In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv


In [2]:
# Import libraries needed

import re #do regex operations
from wordcloud import WordCloud,STOPWORDS 
from sklearn.preprocessing import LabelEncoder

#Visualize
import seaborn as sns
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
#
# metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import f1_score, precision_score, recall_score

# models
from sklearn.svm import SVC

In [3]:
# Import dataset
JobPostDF = pd.read_csv('/kaggle/input/real-or-fake-fake-jobposting-prediction/fake_job_postings.csv')

In [4]:
# Check dataset samples
JobPostDF.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [5]:
JobPostDF.dtypes

job_id                  int64
title                  object
location               object
department             object
salary_range           object
company_profile        object
description            object
requirements           object
benefits               object
telecommuting           int64
has_company_logo        int64
has_questions           int64
employment_type        object
required_experience    object
required_education     object
industry               object
function               object
fraudulent              int64
dtype: object

In [6]:
# Check dataframe shape
JobPostDF.shape

(17880, 18)

In [7]:
# Check for NaN values in the data
JobPostDF.isnull().sum().sort_values(ascending = False).loc[JobPostDF.dtypes == 'object']

salary_range           15012
department             11547
required_education      8105
benefits                7210
required_experience     7050
function                6455
industry                4903
employment_type         3471
company_profile         3308
requirements            2695
location                 346
description                1
title                      0
dtype: int64

In [8]:
# Replace NaN values with a blank entry
JobPostDF.fillna('', inplace = True)

In [9]:
JobPostDF.isnull().sum().sort_values(ascending = False).loc[JobPostDF.dtypes == 'object']

title                  0
function               0
industry               0
required_education     0
required_experience    0
employment_type        0
benefits               0
requirements           0
description            0
company_profile        0
salary_range           0
department             0
location               0
dtype: int64

In [10]:
JobPostDF.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,,,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,0,1,0,,,,,,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",,,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


Comine columns into one for preparation for training

In [11]:
# create copy
JobPostDF_Final = JobPostDF.copy()
JobPostDF_Final.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,,,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,0,1,0,,,,,,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",,,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [12]:
JobPostDF_Final['text-corpus'] = JobPostDF_Final['description'] + ' ' + JobPostDF_Final['requirements']
JobPostDF_Final.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,text-corpus
0,1,Marketing Intern,"US, NY, New York",Marketing,,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,,0,1,0,Other,Internship,,,Marketing,0,"Food52, a fast-growing, James Beard Award-winn..."
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,,Marketing and Advertising,Customer Service,0,Organised - Focused - Vibrant - Awesome!Do you...
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",,,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,,0,1,0,,,,,,0,"Our client, located in Houston, is actively se..."
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,5,Bill Review Manager,"US, FL, Fort Worth",,,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [13]:
for col in JobPostDF_Final.select_dtypes(include=['object']).drop(columns=['text-corpus']):
    JobPostDF_Final.drop(columns=[col], inplace = True)

In [14]:
JobPostDF_Final.head()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent,text-corpus
0,1,0,1,0,0,"Food52, a fast-growing, James Beard Award-winn..."
1,2,0,1,0,0,Organised - Focused - Vibrant - Awesome!Do you...
2,3,0,1,0,0,"Our client, located in Houston, is actively se..."
3,4,0,1,0,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,5,0,1,1,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [15]:
JobPostDF_Final = JobPostDF_Final.drop(columns=['job_id','telecommuting', 'has_company_logo', 'has_questions'])
JobPostDF_Final.head()

,fraudulent,text-corpus
0,0,"Food52, a fast-growing, James Beard Award-winn..."
1,0,Organised - Focused - Vibrant - Awesome!Do you...
2,0,"Our client, located in Houston, is actively se..."
3,0,THE COMPANY: ESRI – Environmental Systems Rese...
4,0,JOB TITLE: Itemization Review ManagerLOCATION:...


In [16]:
JobPostDF_Final['text-corpus'][0]

'Food52, a fast-growing, James Beard Award-winning online food community and crowd-sourced and curated recipe hub, is currently interviewing full- and part-time unpaid interns to work in a small team of editors, executives, and developers in its New York City headquarters.Reproducing and/or repackaging existing Food52 content for a number of partner sites, such as Huffington Post, Yahoo, Buzzfeed, and more in their various content management systemsResearching blogs and websites for the Provisions by Food52 Affiliate ProgramAssisting in day-to-day affiliate program support, such as screening affiliates and assisting in any affiliate inquiriesSupporting with PR &amp; Events when neededHelping with office administrative work, such as filing, mailing, and preparing for meetingsWorking with developers to document bugs and suggest improvements to the siteSupporting the marketing and executive staff Experience with content management systems a major plus (any blogging counts!)Familiar with t

# Data Cleaning

In [17]:
import nltk
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk import tokenize
from nltk.stem import WordNetLemmatizer

In [18]:
# tokenize
def remove(text):
    words = nltk.RegexpTokenizer(r"\w+").tokenize(text)
    
    return words

In [19]:
# remove stopwords function
stop = set(stopwords.words('english'))
def test(text):
    words = remove(text)
    stripped_phrase = []
    for word in words:
        if word not in stop:
            stripped_phrase.append(word)

    return " ".join(stripped_phrase)

In [20]:
# perform stemming function

def stem(text):
    stemmer = PorterStemmer()
    stemmed_phrase = []
    words = remove(text)
    for word in words:
        stemmed_phrase.append(stemmer.stem(word))
    
    return " ".join(stemmed_phrase)

In [21]:
# remove stopwords
JobPostDF_Final['text-corpus'] = JobPostDF_Final['text-corpus'].apply(test)
JobPostDF_Final['text-corpus'][0]

'Food52 fast growing James Beard Award winning online food community crowd sourced curated recipe hub currently interviewing full part time unpaid interns work small team editors executives developers New York City headquarters Reproducing repackaging existing Food52 content number partner sites Huffington Post Yahoo Buzzfeed various content management systemsResearching blogs websites Provisions Food52 Affiliate ProgramAssisting day day affiliate program support screening affiliates assisting affiliate inquiriesSupporting PR amp Events neededHelping office administrative work filing mailing preparing meetingsWorking developers document bugs suggest improvements siteSupporting marketing executive staff Experience content management systems major plus blogging counts Familiar Food52 editorial voice aestheticLoves food appreciates importance home cooking cooking seasonsMeticulous editor perfectionist obsessive attention detail maddened typos broken links delighted finding fixing themChee

In [22]:
# Perform stemming
JobPostDF_Final['text-corpus'] = JobPostDF_Final['text-corpus'].apply(stem)
JobPostDF_Final['text-corpus'][0]

'food52 fast grow jame beard award win onlin food commun crowd sourc curat recip hub current interview full part time unpaid intern work small team editor execut develop new york citi headquart reproduc repackag exist food52 content number partner site huffington post yahoo buzzfe variou content manag systemsresearch blog websit provis food52 affili programassist day day affili program support screen affili assist affili inquiriessupport PR amp event neededhelp offic administr work file mail prepar meetingswork develop document bug suggest improv sitesupport market execut staff experi content manag system major plu blog count familiar food52 editori voic aestheticlov food appreci import home cook cook seasonsmeticul editor perfectionist obsess attent detail madden typo broken link delight find fix themcheer pressureexcel commun skillsa multi tasker juggler respons big smallinterest engag social media like twitter facebook pinterestlov problem solv collabor drive food52 forwardthink big

# Convert to vectors before training

In [23]:
# from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# corpus = JobPostDF_Final['text-corpus'].values
# corpus

# # create vectorizer
# vectorizer = CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(2,3))

# # Converting the text to numeric data
# X = vectorizer.fit_transform(corpus)

# # add to new dataframe
# CountVectorizedData=pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out())
# CountVectorizedData['fraudulent'] = JobPostDF_Final['fraudulent']
# CountVectorizedData.head()

# Extract features

In [24]:
# Get independent(X) features
X = JobPostDF_Final['text-corpus']
X.shape

(17880,)

In [25]:
# Show head
X.head()

0    food52 fast grow jame beard award win onlin fo...
1    organis focus vibrant awesom Do passion custom...
2    our client locat houston activ seek experienc ...
3    the compani esri environment system research i...
4    job titl item review managerloc fort worth TX ...
Name: text-corpus, dtype: object

In [26]:
# Get Dependent(Y) features
Y = JobPostDF_Final['fraudulent']
Y.shape

(17880,)

In [27]:
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [28]:
encoder = LabelEncoder()
train_y = encoder.fit_transform(train_y)
test_y = encoder.fit_transform(test_y)

In [29]:
# transform data to count matrix
MAX = 3000

In [30]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# tfid = TfidfVectorizer(min_df=0, max_df=1, ngram_range=(1,3))
tfid = TfidfVectorizer(max_features = MAX,use_idf=True)
# cv=CountVectorizer(min_df=0,max_df=1,binary=False,ngram_range=(2,3))

# Transform train set
tfid_train = tfid.fit_transform(train_X)
# cv_train = cv.fit_transform(train_X)

# Transform test set
tfid_test = tfid.fit_transform(test_X)
# cv_test = cv.fit_transform(test_X)

print('Tfidf_train : ', tfid_train.shape)
# print('cv_train : ', cv_train.shape)

print('Tfidf_test : ', tfid_test.shape)
# print('cv_test : ', cv_test.shape)


Tfidf_train :  (14304, 3000)
Tfidf_test :  (3576, 3000)


# Modelling

Naive Bayes

In [31]:
from sklearn.naive_bayes import GaussianNB, MultinomialNB
# set the model
nb_model = MultinomialNB()
# nb_model = GaussianNB()

# fit the dataset
nb_tfidf = nb_model.fit(tfid_train, train_y)
print(nb_tfidf)

MultinomialNB()


# Evaluate model

In [32]:
nb_tfidf_predict = nb_model.predict(tfid_test)

In [33]:
#Accuracy score for tfidf features

nb_tfidf_score=accuracy_score(test_y, nb_tfidf_predict)
print("Accuracy  {:.3} %".format(accuracy_score(test_y, nb_tfidf_predict)*100))

Accuracy  85.9 %


In [34]:
report = classification_report(test_y, nb_tfidf_predict, target_names = ['0', '1'])
print(report)

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3423
           1       0.01      0.03      0.02       153

    accuracy                           0.86      3576
   macro avg       0.48      0.46      0.47      3576
weighted avg       0.91      0.86      0.89      3576



# Using SVM model

In [35]:
# fit data using SVC
svc = SVC(kernel='rbf')
svc.fit(tfid_train, train_y)

SVC()

In [36]:
# predict output of the test data set
predicted = svc.predict(tfid_test)

In [37]:
print("accuracy score of Support Vector Machine:", accuracy_score(predicted, test_y), "\n")

accuracy score of Support Vector Machine: 0.9572147651006712 



In [38]:
# creating a confusion matrix 
cm = confusion_matrix(test_y, predicted)
print("confusion matrix of SVM:\n", cm, "\n")

confusion matrix of SVM:
 [[3423    0]
 [ 153    0]] 



In [39]:
# create a classifcation report
print("classification report:\n", classification_report(test_y, predicted), "\n")

classification report:
               precision    recall  f1-score   support

           0       0.96      1.00      0.98      3423
           1       0.00      0.00      0.00       153

    accuracy                           0.96      3576
   macro avg       0.48      0.50      0.49      3576
weighted avg       0.92      0.96      0.94      3576
 



/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# Using BiLSTM

In [40]:
import tensorflow
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

In [41]:
df = JobPostDF[['description','requirements','fraudulent']]
df.head()

,description,requirements,fraudulent
0,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,0
1,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,0
2,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,0
3,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",0
4,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,0


In [42]:
df.shape

(17880, 3)

In [43]:
###Drop Nan Values
df=df.dropna()

In [44]:
df.shape

(17880, 3)

In [45]:
## Get the Independent Features

new_X=df.drop('fraudulent',axis=1)
new_X.head()

,description,requirements
0,"Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...
1,Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...
2,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...
3,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi..."
4,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...


In [46]:
## Get the Dependent features
new_y=df['fraudulent']
new_y.head()

0    0
1    0
2    0
3    0
4    0
Name: fraudulent, dtype: int64

In [47]:
new_y.value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

In [48]:
voc_size=5000

# One hot representation

In [49]:
message = new_X.copy()

In [50]:
message['description'][1]

"Organised - Focused - Vibrant - Awesome!Do you have a passion for customer service? Slick typing skills? Maybe Account Management? ...And think administration is cooler than a polar bear on a jetski? Then we need to hear you!\xa0We are the Cloud Video Production Service and opperating on a glodal level. Yeah, it's pretty cool. Serious about\xa0delivering a world class product and excellent customer service.Our rapidly expanding business is looking for a talented Project Manager to manage the successful delivery of video projects, manage client communications and drive the production process. Work with some of the coolest brands on the planet and learn from a global team that are representing NZ is a huge way!We are entering the next growth stage of our business and growing quickly internationally. \xa0Therefore, the position is bursting with opportunity for the right person entering the business at the right time.\xa090 Seconds, the worlds Cloud Video Production Service -\xa0http://90

In [51]:
message.reset_index(inplace=True)

In [52]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(message)):
    review = re.sub('[^a-zA-Z]', ' ', message['description'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [53]:
corpus[1]

'organis focus vibrant awesom passion custom servic slick type skill mayb account manag think administr cooler polar bear jetski need hear cloud video product servic opper glodal level yeah pretti cool seriou deliv world class product excel custom servic rapidli expand busi look talent project manag manag success deliveri video project manag client commun drive product process work coolest brand planet learn global team repres nz huge way enter next growth stage busi grow quickli intern therefor posit burst opportun right person enter busi right time second world cloud video product servic http url fbe afac cd c f b eef e e f ca dd second world cloud video product servic enabl brand agenc get high qualiti onlin video content shot produc anywher world fast afford manag seamlessli cloud purchas publish second remov hassl cost risk speed issu work regular video product compani manag everi aspect video project beauti onlin experi grow network rate video profession countri dedic product suc

In [54]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr[1]

[1772,
 3719,
 627,
 3904,
 1386,
 2741,
 859,
 26,
 687,
 94,
 2115,
 2545,
 2361,
 1207,
 3314,
 279,
 4460,
 2030,
 2247,
 2147,
 1189,
 563,
 2110,
 1879,
 859,
 945,
 1713,
 749,
 194,
 671,
 4795,
 4656,
 4950,
 2468,
 633,
 1879,
 3952,
 2741,
 859,
 810,
 1428,
 1923,
 452,
 3695,
 3317,
 2361,
 2361,
 4485,
 3434,
 2110,
 3317,
 2361,
 3664,
 2868,
 747,
 1879,
 137,
 652,
 4217,
 2261,
 757,
 1615,
 2433,
 1980,
 814,
 452,
 120,
 3463,
 266,
 4130,
 2866,
 703,
 1923,
 4457,
 264,
 3642,
 2745,
 3611,
 2222,
 3375,
 1948,
 1342,
 266,
 1923,
 1948,
 2391,
 3945,
 2468,
 563,
 2110,
 1879,
 859,
 3167,
 2836,
 3130,
 2780,
 4456,
 1574,
 1364,
 1350,
 4183,
 836,
 836,
 1364,
 474,
 377,
 3945,
 2468,
 563,
 2110,
 1879,
 859,
 10,
 2261,
 1705,
 625,
 4333,
 918,
 4344,
 2110,
 3644,
 2513,
 4302,
 2380,
 2468,
 3382,
 1182,
 2361,
 4375,
 563,
 3307,
 3134,
 3945,
 4736,
 3305,
 4983,
 3582,
 779,
 2607,
 652,
 3378,
 2110,
 1879,
 1113,
 2361,
 3762,
 864,
 2110,
 3317,
 3

#  Embedding Representation:


In [55]:
sent_length=40
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[4925 3644 2361 ... 1775  683 4597]
 [4485 2409 2525 ...  557 3964 1002]
 [   0    0    0 ... 3620 3334 2890]
 ...
 [2335 3078 4650 ... 3594 4063 4933]
 [4315 2526 4839 ... 3798 3642 3317]
 [2447 1811 2044 ... 3464  877 2875]]


In [56]:
embedded_docs[0]

array([4925, 3644, 2361, 2534, 1246,  207, 3890, 1609, 4229, 3039, 2602,
       2602, 4229,  835,  672, 3935, 4229, 4580, 4229, 1547, 3363, 1429,
       3878, 3224,  813, 3314,  652, 1686,  909, 2871, 3282, 3326, 4112,
       1930,  366, 4099, 1453, 1775,  683, 4597], dtype=int32)

In [57]:
## Creating model
embedding_vector_features=50
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100))) ##Just add bidirectional!!, except it would just behave as normal LSTM Model
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

2022-07-21 11:23:48.401871: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 11:23:48.502675: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 11:23:48.503496: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-21 11:23:48.504908: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 40, 50)            250000    
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               120800    
_________________________________________________________________
dropout (Dropout)            (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 371,001
Trainable params: 371,001
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
len(embedded_docs),new_y.shape

(17880, (17880,))

In [59]:
X_final=np.array(embedded_docs)
y_final=np.array(new_y)

In [60]:
X_final[1]

array([4485, 2409, 2525, 2061, 3287,  972, 1071, 3945, 4302,  935, 2110,
       3760, 2433, 2261, 1375, 2468, 1786, 1375, 3938,  991, 1084, 3905,
       1982,  813,  899, 4098, 1849, 4374, 1429,  702,  899,  813, 2666,
       1948, 2910, 1490,   29,  557, 3964, 1002], dtype=int32)

In [61]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.25, random_state=32)

In [62]:
model1.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=12,batch_size=64)

2022-07-21 11:23:51.531330: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/12


2022-07-21 11:23:55.232654: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


210/210 [==============================] - 7s 11ms/step - loss: 0.1900 - accuracy: 0.9540 - val_loss: 0.1256 - val_accuracy: 0.9644
Epoch 2/12
210/210 [==============================] - 2s 9ms/step - loss: 0.0944 - accuracy: 0.9729 - val_loss: 0.1112 - val_accuracy: 0.9689
Epoch 3/12
210/210 [==============================] - 2s 9ms/step - loss: 0.0594 - accuracy: 0.9817 - val_loss: 0.1028 - val_accuracy: 0.9732
Epoch 4/12
210/210 [==============================] - 2s 8ms/step - loss: 0.0389 - accuracy: 0.9887 - val_loss: 0.1094 - val_accuracy: 0.9720
Epoch 5/12
210/210 [==============================] - 2s 8ms/step - loss: 0.0254 - accuracy: 0.9922 - val_loss: 0.1231 - val_accuracy: 0.9738
Epoch 6/12
210/210 [==============================] - 2s 8ms/step - loss: 0.0133 - accuracy: 0.9961 - val_loss: 0.1607 - val_accuracy: 0.9738
Epoch 7/12
210/210 [==============================] - 2s 8ms/step - loss: 0.0072 - accuracy: 0.9986 - val_loss: 0.1843 - val_accuracy: 0.9747
Epoch 8/12
210/2

# Model Performance and Accuracy:


In [63]:
y_pred=model1.predict(X_test)
y_pred

array([[6.3997260e-07],
       [5.9522154e-07],
       [5.3688109e-07],
       ...,
       [9.9999118e-01],
       [2.1392701e-07],
       [1.2854945e-03]], dtype=float32)

In [64]:
y_pred = [1 if x >=0.5 else 0 for x in y_pred]
y_pred[5:20]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [65]:
confusion_matrix(y_test,y_pred)

array([[4218,   29],
       [  83,  140]])

In [66]:
accuracy_score(y_test,y_pred)
print("Accuracy  {:.3} %".format(accuracy_score(y_test, y_pred)*100))

Accuracy  97.5 %


In [67]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      4247
           1       0.83      0.63      0.71       223

    accuracy                           0.97      4470
   macro avg       0.90      0.81      0.85      4470
weighted avg       0.97      0.97      0.97      4470



# Therefore,after comparing results:

In [68]:
#Accuracy score for Naive Bayes features

nb_tfidf_score=accuracy_score(test_y, nb_tfidf_predict)
print("Naive Bayes Accuracy  {:.3} %".format(accuracy_score(test_y, nb_tfidf_predict)*100))

Naive Bayes Accuracy  85.9 %


In [69]:
print("Support Vector Machine Accuracy  {:.3} %".format(accuracy_score(predicted, test_y)*100))

Support Vector Machine Accuracy  95.7 %


In [70]:
print("BiLSTM Accuracy  {:.3} %".format(accuracy_score(y_test, y_pred)*100))
print(classification_report(y_test,y_pred))

BiLSTM Accuracy  97.5 %
              precision    recall  f1-score   support

           0       0.98      0.99      0.99      4247
           1       0.83      0.63      0.71       223

    accuracy                           0.97      4470
   macro avg       0.90      0.81      0.85      4470
weighted avg       0.97      0.97      0.97      4470

